In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Get_file import Get_file   # return dirpath, dirnames, filenames
from FFT import FFT, PlotDataOnTimeDomain
from BandPassFilter import BandPassFilter, PlotDataOnFreqDomain, BandPassListFilter
from Danny_ML_CLF import Danny_ML_CLF
import os

%matplotlib inline

data_dir = "C:\\Users\\zxpay\\Desktop\\BME_Senior\\EEG_CLF\\Emotion\\database\\New_SDK_Data"


C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def CalculateZeros(NumpyArray):
    LogicWithZeros = (NumpyArray == 0)
    return np.sum(LogicWithZeros)

In [3]:
os.chdir(data_dir)
_,data_files,_ = Get_file(data_dir)
data_files
all_df = pd.DataFrame()
for file in data_files:
    emotion = 0
    # 1->normal, 2->happy, 3->scare, 4->touch, 
    _,_,file_names = Get_file(data_dir + '\\' + file)
    for fn in file_names:
        
        data_name = data_dir + '\\' + file + '\\' + fn
        if 'happy' in fn:
            emotion = 2
        elif 'scare' in fn:
            emotion = 3
        elif 'touch' in fn:
            emotion = 4
        else:
            print(fn)
            emotion = 1
        #print(emotion, fn)
        df = pd.read_csv(data_name, engine='python')
        df_length = df.shape[0]
        
        df['label'] = df['label']*emotion
            
        try:
            all_df = pd.concat([all_df,df])
        except Exception as e:
            print(e)
            all_df = df
        #print(sorted(df.corr()['label'].tolist()), fn)
        #print('***********************************************')
        

In [4]:
# 移除遺失值
all_df = all_df.dropna(axis=1)
all_df = all_df.reset_index()

In [5]:
Columns = ['alpha', 'channel', 'gamma', 'heig_beta', 'label','low_beta','theta','time']
Features = ['alpha', 'heig_beta', 'low_beta','theta']
Labels = ['label']

In [6]:
channel1 = all_df[all_df['channel'] == 1].index
channel2 = all_df[all_df['channel'] == 2].index
channel3 = all_df[all_df['channel'] == 3].index
channel4 = all_df[all_df['channel'] == 4].index
channel5 = all_df[all_df['channel'] == 5].index
channel6 = all_df[all_df['channel'] == 6].index
channel7 = all_df[all_df['channel'] == 7].index
channel8 = all_df[all_df['channel'] == 8].index
channel9 = all_df[all_df['channel'] == 9].index
channel10 = all_df[all_df['channel'] == 10].index
channel11 = all_df[all_df['channel'] == 11].index
channel12 = all_df[all_df['channel'] == 12].index
channel13 = all_df[all_df['channel'] == 13].index
channel14 = all_df[all_df['channel'] == 14].index

df_1 = all_df.iloc[channel1]
df_2 = all_df.iloc[channel2]
df_3 = all_df.iloc[channel3]
df_4 = all_df.iloc[channel4]
df_5 = all_df.iloc[channel5]
df_6 = all_df.iloc[channel6]
df_7 = all_df.iloc[channel7]
df_8 = all_df.iloc[channel8]
df_9 = all_df.iloc[channel9]
df_10 = all_df.iloc[channel10]
df_11 = all_df.iloc[channel11]
df_12 = all_df.iloc[channel12]
df_13 = all_df.iloc[channel13]
df_14 = all_df.iloc[channel14]


In [7]:
my_df = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9,
        df_10, df_11, df_12, df_13, df_14]

In [8]:
os.chdir('C:\\Users\\zxpay\\Desktop\\BME_Senior\\EEG_CLF\\Emotion\\Code')
for n in range(len(my_df)):
    n=9
    Testing_df = my_df[n]
    data = np.zeros([Testing_df.shape[0], 4])
    for i in range(4):
        data[:, i] = np.array(Testing_df[Features[i]], dtype=np.float32)
    label = np.array(Testing_df['label'], dtype=np.float32)
    if CalculateZeros(data) != 0:
        print('Warning !!!')

    pos = np.where((label==0) | (label==2))
    
    zero_pos = np.where(label==0)
    zero_choose = np.random.randint(500,20000,size=400)
    zero_choose = zero_pos[0][zero_choose]
    data_normal = data[zero_choose]
    label_normal = label[zero_choose]
    
    pos_happy = np.where(label==2)
    one_choose = np.random.randint(20,1020,size=400)
    one_choose = pos_happy[0][one_choose]
    data_happy = data[one_choose]
    label_happy = label[one_choose]

    data_del = np.delete(data, pos, axis=0)
    label_del = np.delete(label, pos)

    data_all = np.vstack((data_normal, data_del))
    label_all = np.hstack((label_normal, label_del))
    data_all = np.vstack((data_happy, data_all))
    label_all = np.hstack((label_happy, label_all))

    print(data_all.shape)
    print(label_all.shape)


    clf = Danny_ML_CLF()
    train_X, test_X, train_y, test_y = clf.Split_data(data_all, label_all, test_size=0.3, Standard=True)
    clf.Fit_value(train_X, train_y)
    clf.Train()
    clf.Report(train_X, train_y, [0,2,3,4], show_cm=False)
    clf.Report2txt('Train' + str(n+1) + '.txt')
    print(clf.Score())
    print()
    clf.Report(test_X, test_y, [0,2,3,4], show_cm=False)
    clf.Report2txt('Test' + str(n+1) + '.txt')
    print(clf.Score())
    print('Report Okay !\n\n')

(1531, 4)
(1531,)


C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

SVM Score :  0.4827264239028945
Tree Score :  0.5452847805788982
Bayes Score :  0.35947712418300654
KNN Score :  1.0
XGB Score :  0.715219421101774
Stacking Score :  0.9561157796451915
Voting Score :  0.9495798319327731
Bagging Score :  1.0
RF Score :  1.0
Adaboost Score :  0.5256769374416433
None



C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in 

SVM Score :  0.46956521739130436
Tree Score :  0.46304347826086956
Bayes Score :  0.3717391304347826
KNN Score :  0.45652173913043476
XGB Score :  0.5021739130434782
Stacking Score :  0.44130434782608696
Voting Score :  0.4934782608695652
Bagging Score :  0.5260869565217391
RF Score :  0.5173913043478261
Adaboost Score :  0.45
None
Report Okay !


(1531, 4)
(1531,)


...

C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 0.4434782608695652
Tree Score :  0.46956521739130436
Bayes Score :  0.3630434782608696
KNN Score :  0.47608695652173916
XGB Score :  0.4956521739130435
Stacking Score :  0.44782608695652176
Voting Score :  0.49782608695652175
Bagging Score :  0.49130434782608695
RF Score :  0.5043478260869565
Adaboost Score :  0.4434782608695652
None
Report Okay !


(1531, 4)
(1531,)


C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

SVM Score :  0.480859010270775
Tree Score :  0.5415499533146592
Bayes Score :  0.39775910364145656
KNN Score :  1.0
XGB Score :  0.7068160597572363
Stacking Score :  0.9337068160597572
Voting Score :  0.9393090569561158
Bagging Score :  1.0
RF Score :  1.0
Adaboost Score :  0.5322128851540616
None

SVM Score :  0.508695652173913
Tree Score :  0.4717391304347826
Bayes Score :  0.3760869565217391
KNN Score :  0.49782608695652175
XGB Score :  0.5108695652173914
Stacking Score :  0.48695652173913045
Voting Score :  0.5195652173913043
Bagging Score :  0.49782608695652175
RF Score :  0.5130434782608696
Adaboost Score :  0.4652173913043478
None
Report Okay !




C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

In [9]:
np.sum(clf.RF_prediction(test_X)==test_y)/test_y.shape[0]

0.5130434782608696

In [10]:
print(np.sum(label_all==0))
print(np.sum(label_all==2))
print(np.sum(label_all==3))
print(np.sum(label_all==4))

400
400
351
380


In [11]:
clf.RF_prediction(test_X)

array([4., 4., 3., 0., 3., 3., 2., 3., 2., 2., 2., 4., 3., 4., 3., 3., 2.,
       3., 4., 0., 3., 0., 4., 3., 0., 3., 3., 4., 4., 4., 3., 2., 2., 3.,
       2., 3., 3., 2., 3., 3., 0., 2., 2., 2., 2., 2., 2., 4., 3., 4., 0.,
       2., 3., 3., 4., 0., 4., 0., 4., 3., 4., 0., 4., 4., 4., 4., 0., 2.,
       3., 0., 2., 2., 4., 3., 4., 4., 4., 4., 3., 4., 0., 4., 2., 3., 0.,
       3., 2., 2., 0., 0., 4., 3., 2., 2., 3., 3., 3., 4., 4., 4., 4., 0.,
       0., 4., 4., 2., 0., 0., 0., 4., 0., 2., 3., 3., 2., 3., 4., 2., 4.,
       4., 3., 4., 2., 2., 3., 0., 4., 0., 0., 2., 3., 3., 3., 2., 3., 2.,
       2., 4., 3., 0., 2., 2., 0., 2., 2., 3., 0., 0., 4., 3., 2., 2., 3.,
       4., 3., 2., 2., 3., 4., 4., 2., 4., 3., 4., 4., 4., 3., 0., 2., 0.,
       4., 0., 2., 4., 4., 3., 0., 2., 3., 0., 2., 2., 3., 2., 3., 0., 0.,
       2., 0., 4., 4., 4., 4., 2., 4., 4., 0., 4., 4., 2., 2., 3., 0., 2.,
       2., 2., 4., 2., 2., 4., 4., 0., 2., 2., 0., 4., 0., 2., 4., 4., 3.,
       4., 4., 3., 3., 0.

In [12]:
from sklearn.tree import DecisionTreeClassifier
label_name = []
for x in train_y:
    if x == 0:
        label_name.append('normal')
    elif x == 2:
        label_name.append('happy')
    elif x == 3:
        label_name.append('scare')
    elif x == 4:
        label_name.append('touch')
    else:
        print(x)
label_name_test = []
for x in test_y:
    if x == 0:
        label_name_test.append('normal')
    elif x == 2:
        label_name_test.append('happy')
    elif x == 3:
        label_name_test.append('scare')
    elif x == 4:
        label_name_test.append('touch')
    else:
        print(x)
tree = DecisionTreeClassifier(criterion='gini',max_depth=3)
tree.fit(train_X, label_name)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [13]:
from sklearn.tree import export_graphviz
export_graphviz(tree, out_file="tree.dot", feature_names=['alpha', 'heig_beta', 'low_beta','theta'],\
                class_names=['normal','happy','scare', 'touch'])


In [14]:
!dot -Tpng tree.dot -o tree.png

In [15]:
tree.predict(test_X)

array(['scare', 'normal', 'scare', 'normal', 'scare', 'normal', 'happy',
       'scare', 'happy', 'happy', 'happy', 'touch', 'normal', 'scare',
       'scare', 'scare', 'happy', 'scare', 'touch', 'scare', 'scare',
       'normal', 'normal', 'scare', 'scare', 'touch', 'scare', 'normal',
       'touch', 'normal', 'normal', 'happy', 'happy', 'touch', 'scare',
       'scare', 'scare', 'happy', 'scare', 'scare', 'normal', 'happy',
       'happy', 'scare', 'happy', 'happy', 'happy', 'scare', 'touch',
       'touch', 'normal', 'happy', 'normal', 'normal', 'touch', 'touch',
       'touch', 'scare', 'normal', 'normal', 'normal', 'normal', 'normal',
       'normal', 'touch', 'touch', 'touch', 'scare', 'touch', 'normal',
       'happy', 'happy', 'normal', 'normal', 'touch', 'scare', 'scare',
       'touch', 'normal', 'normal', 'normal', 'normal', 'happy', 'normal',
       'normal', 'normal', 'happy', 'happy', 'normal', 'normal', 'normal',
       'scare', 'happy', 'scare', 'normal', 'scare', 'norm

In [16]:
tree.score(test_X, label_name_test)

0.45

In [17]:
pos = np.where((label==0) | (label==2))

In [18]:
pos

(array([    0,     1,     2, ..., 47066, 47067, 47068], dtype=int64),)